<h1>
<center>
Predicting Rain in Australia
</center>
</h1>
<h3>
<center>
Sean Wilson (CIS 399)
</center>
</h3>

In [1]:
import pandas as pd
import numpy as np
import re
import os

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/EXTRA_ASS/'

file_name = 'weatherAUS.csv'

weather_table = pd.read_csv(home_path + file_path + file_name)

In [2]:
from library_w19_deep_2 import *

%who function

ann_flex	 ann_flex_predictor	 ann_flex_tester	 ann_predictor	 ann_simple	 ann_tester	 from_scratch	 from_scratch_flex	 mse	 
mse_der	 sigmoid	 sigmoid_der	 


<h3>

want to see the null value dispersion among all columns

</h3>

In [3]:
weather_table.isna().sum().sort_values(ascending=False)

Sunshine         67816
Evaporation      60843
Cloud3pm         57094
Cloud9am         53657
Pressure9am      14014
Pressure3pm      13981
WindDir9am       10013
WindGustDir       9330
WindGustSpeed     9270
WindDir3pm        3778
Humidity3pm       3610
Temp3pm           2726
WindSpeed3pm      2630
Humidity9am       1774
RainToday         1406
Rainfall          1406
WindSpeed9am      1348
Temp9am            904
MinTemp            637
MaxTemp            322
Location             0
RainTomorrow         0
RISK_MM              0
Date                 0
dtype: int64

<h3>

from finding the nulls, I've decided to drop columns Cloud9am, Cloud3pm, Evaporation, and Sunshine completely from the table.

</h3>

In [4]:
weather_table.drop(columns={"Cloud9am", "Cloud3pm", "Evaporation", "Sunshine", "RISK_MM"}, inplace=True)

<h3>

drop null rows

</h3>

In [5]:
weather_table = weather_table.dropna(how="any")

print(weather_table.size)

weather_table.isna().sum().sort_values(ascending=False)

2145575


RainTomorrow     0
WindDir3pm       0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindSpeed9am     0
RainToday        0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
Date             0
dtype: int64

<h3>

testing using regex to strip non-int characters from date column

</h3>

In [6]:
import re

test = "l1998-12-31"
test = re.sub("[^0-9]", "", test)
print(test)
test = test[-1] + test[:-1]
print(test)

19981231
11998123


<h3>

more testing...

</h3>

In [7]:
test = "hvfe200;wf8-12-31oujlrbwg"
print(test)
test = re.sub("[^0-9,-]", "", test)
print(test)
test = test.split('-')
print(test)

year = test[0]
mon = test[1]
day = test[2]

year = int(year)
print(year)
mon = int(mon)
print(mon)
day = int(day)
print(day)

hvfe200;wf8-12-31oujlrbwg
2008-12-31
['2008', '12', '31']
2008
12
31


<h3>

from the testing I did above, I'm going to create functions that I can use on every row to get the date formated

</h3>

In [8]:
#eliminates any extra characters from date
def date_format(date_str):
    test = re.sub("[^0-9,-]", "", date_str)
    return test

#grab the year, turn it into int
def year_f(date_str):
    test = date_str.split('-')
    year = test[0]
    return int(year)
    
#grab the month, turn it into int
def mon_f(date_str):
    test = date_str.split('-')
    mon = test[1]
    return int(mon)

#I left off grabbing the day because I think it would be irrelevant data

#format
weather_table['Date'].apply(date_format)

#grab/make new columns
weather_table['Year'] = weather_table['Date'].apply(year_f)
weather_table['Month'] = weather_table['Date'].apply(mon_f)

<h3>

looking at all the different waether stations. I think I'm going to pick 1 to focus on
</h3>

In [9]:
weather_table.Location.unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'NorahHead', 'NorfolkIsland', 'Richmond', 'Sydney',
       'SydneyAirport', 'WaggaWagga', 'Williamtown', 'Wollongong',
       'Canberra', 'Tuggeranong', 'Ballarat', 'Bendigo', 'Sale',
       'MelbourneAirport', 'Melbourne', 'Mildura', 'Nhil', 'Portland',
       'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns', 'GoldCoast',
       'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa', 'Woomera',
       'Witchcliffe', 'PearceRAAF', 'PerthAirport', 'Perth', 'Walpole',
       'Hobart', 'Launceston', 'AliceSprings', 'Darwin', 'Katherine',
       'Uluru'], dtype=object)

<h3>

Limiting the df to one location
</h3>

In [10]:
# *******************************

weather_table = weather_table[weather_table.Location == 'Darwin']

# *******************************

<h3>

142192 rows -> 3117 rows
</h3>

In [11]:
weather_table.drop(columns={"Location", "Date"}, inplace=True)

print(weather_table.shape)

weather_table.head()

(3117, 19)


,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
135921,20.0,33.1,0.0,E,41.0,ENE,SSE,13.0,17.0,81.0,32.0,1016.0,1012.1,25.4,32.3,No,No,2008,7
135922,19.4,32.4,0.0,ENE,50.0,SE,E,15.0,28.0,81.0,17.0,1016.8,1012.4,24.3,31.9,No,No,2008,7
135923,18.2,31.8,0.0,E,46.0,ESE,ENE,22.0,19.0,38.0,24.0,1017.2,1013.0,24.3,31.2,No,No,2008,7
135924,17.3,30.7,0.0,E,44.0,SE,E,22.0,13.0,55.0,16.0,1017.3,1013.6,21.3,29.8,No,No,2008,7
135925,15.5,30.8,0.0,ESE,46.0,E,E,20.0,19.0,37.0,16.0,1016.3,1012.6,22.2,29.6,No,No,2008,7


In [12]:
weather_table.WindGustDir.unique()

array(['E', 'ENE', 'ESE', 'NE', 'WNW', 'N', 'NW', 'SE', 'NNW', 'SSE',
       'NNE', 'S', 'W', 'WSW', 'SSW', 'SW'], dtype=object)

<h3>

time to wrangle the wind columns with a function
</h3>

In [13]:
def wind_bender(w_dir):
    if w_dir == 'N':
        return 1
    elif w_dir == 'NNE':
        return 2
    elif w_dir == 'NE':
        return 3
    elif w_dir == 'ENE':
        return 4
    elif w_dir == 'E':
        return 5
    elif w_dir == 'ESE':
        return 6
    elif w_dir == 'SE':
        return 7
    elif w_dir == 'SSE':
        return 8
    elif w_dir == 'S':
        return 9
    elif w_dir == 'SSW':
        return 10
    elif w_dir == 'SW':
        return 11
    elif w_dir == 'WSW':
        return 12
    elif w_dir == 'W':
        return 13
    elif w_dir == 'WNW':
        return 14
    elif w_dir == 'NW':
        return 15
    else:# w_dir == 'NNW':
        return 16

<h3>

applying the above function
</h3>

In [14]:
weather_table['WindGustDir'] = weather_table['WindGustDir'].apply(wind_bender)
weather_table['WindDir9am'] = weather_table['WindDir9am'].apply(wind_bender)
weather_table['WindDir3pm'] = weather_table['WindDir3pm'].apply(wind_bender)

weather_table.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
135921,20.0,33.1,0.0,5,41.0,4,8,13.0,17.0,81.0,32.0,1016.0,1012.1,25.4,32.3,No,No,2008,7
135922,19.4,32.4,0.0,4,50.0,7,5,15.0,28.0,81.0,17.0,1016.8,1012.4,24.3,31.9,No,No,2008,7
135923,18.2,31.8,0.0,5,46.0,6,4,22.0,19.0,38.0,24.0,1017.2,1013.0,24.3,31.2,No,No,2008,7
135924,17.3,30.7,0.0,5,44.0,7,5,22.0,13.0,55.0,16.0,1017.3,1013.6,21.3,29.8,No,No,2008,7
135925,15.5,30.8,0.0,6,46.0,5,5,20.0,19.0,37.0,16.0,1016.3,1012.6,22.2,29.6,No,No,2008,7


In [15]:
weather_table.RainToday.unique()

array(['No', 'Yes'], dtype=object)

In [16]:
weather_table['RainToday'] = weather_table.apply(lambda row: 0 if (row.RainToday == 'No') else 1, axis=1)
weather_table['RainTomorrow'] = weather_table.apply(lambda row: 0 if (row.RainTomorrow == 'No') else 1, axis=1)

<h3>

Now that I have all my values in my table, time to normalize EVERYTHING
</h3>

In [17]:
weather_table["MinTemp"] = weather_table["MinTemp"] / weather_table["MinTemp"].max()

weather_table["MaxTemp"] = weather_table["MaxTemp"] / weather_table["MaxTemp"].max()

weather_table["WindGustSpeed"] = weather_table["WindGustSpeed"] / weather_table["WindGustSpeed"].max()

weather_table["WindSpeed9am"] = weather_table["WindSpeed9am"] / weather_table["WindSpeed9am"].max()

weather_table["WindSpeed3pm"] = weather_table["WindSpeed3pm"] / weather_table["WindSpeed3pm"].max()

weather_table["Humidity9am"] = weather_table["Humidity9am"] / weather_table["Humidity9am"].max()

weather_table["Humidity3pm"] = weather_table["Humidity3pm"] / weather_table["Humidity3pm"].max()

weather_table["Pressure9am"] = weather_table["Pressure9am"] / weather_table["Pressure9am"].max()

weather_table["Pressure3pm"] = weather_table["Pressure3pm"] / weather_table["Pressure3pm"].max()

weather_table["Temp9am"] = weather_table["Temp9am"] / weather_table["Temp9am"].max()

weather_table["Temp3pm"] = weather_table["Temp3pm"] / weather_table["Temp3pm"].max()


weather_table["Rainfall"] = weather_table["Rainfall"] / weather_table["Rainfall"].max()

# for the date, I first subtract it from the max date to get a number closer to 0, with 0 being the max year
weather_table["Year"] = weather_table["Year"].max() - weather_table["Year"]
weather_table["Year"] = weather_table["Year"] / weather_table["Year"].max()

weather_table["Month"] = weather_table["Month"] / weather_table["Month"].max()


weather_table["WindGustDir"] = weather_table["WindGustDir"] / weather_table["WindGustDir"].max()
weather_table["WindDir9am"] = weather_table["WindDir9am"] / weather_table["WindDir9am"].max()
weather_table["WindDir3pm"] = weather_table["WindDir3pm"] / weather_table["WindDir3pm"].max()


weather_table.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
135921,0.673401,0.871053,0.0,0.3125,0.325397,0.2500,0.5000,0.206349,0.326923,0.818182,0.323232,0.996762,0.996750,0.788820,0.863636,0,0,1.0,0.583333
135922,0.653199,0.852632,0.0,0.2500,0.396825,0.4375,0.3125,0.238095,0.538462,0.818182,0.171717,0.997547,0.997045,0.754658,0.852941,0,0,1.0,0.583333
135923,0.612795,0.836842,0.0,0.3125,0.365079,0.3750,0.2500,0.349206,0.365385,0.383838,0.242424,0.997940,0.997636,0.754658,0.834225,0,0,1.0,0.583333
135924,0.582492,0.807895,0.0,0.3125,0.349206,0.4375,0.3125,0.349206,0.250000,0.555556,0.161616,0.998038,0.998227,0.661491,0.796791,0,0,1.0,0.583333
135925,0.521886,0.810526,0.0,0.3750,0.365079,0.3125,0.3125,0.317460,0.365385,0.373737,0.161616,0.997057,0.997242,0.689441,0.791444,0,0,1.0,0.583333


<h3>
what the year values are now. 0 = 2008, 1 = 2017
</h3>

In [18]:
weather_table.Year.unique()

array([1.        , 0.88888889, 0.77777778, 0.66666667, 0.55555556,
       0.44444444, 0.33333333, 0.22222222, 0.11111111, 0.        ])

<h3>
looking ok so far. Time to test.
</h3>

In [19]:
weather_table.describe()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
count,3117.000000,3117.000000,3117.000000,3117.00000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000
mean,0.780612,0.856367,0.013678,0.51155,0.321962,0.464750,0.618022,0.240626,0.401317,0.690009,0.521192,0.992772,0.993087,0.843722,0.831345,0.263073,0.262111,0.500018,0.542910
std,0.100173,0.045998,0.044898,0.31502,0.079524,0.245753,0.344326,0.107608,0.115369,0.152151,0.188149,0.003289,0.002866,0.084943,0.055612,0.440373,0.439853,0.295138,0.283678
min,0.420875,0.576316,0.000000,0.06250,0.119048,0.062500,0.062500,0.031746,0.076923,0.111111,0.050505,0.977435,0.980894,0.518634,0.508021,0.000000,0.000000,0.000000,0.083333
25%,0.727273,0.831579,0.000000,0.31250,0.261905,0.312500,0.312500,0.174603,0.326923,0.626263,0.393939,0.990582,0.991235,0.791925,0.799465,0.000000,0.000000,0.222222,0.333333
50%,0.804714,0.863158,0.000000,0.37500,0.309524,0.437500,0.812500,0.238095,0.384615,0.717172,0.535354,0.993133,0.993402,0.863354,0.842246,0.000000,0.000000,0.444444,0.583333
75%,0.851852,0.886842,0.004353,0.87500,0.365079,0.625000,0.937500,0.301587,0.461538,0.787879,0.646465,0.995193,0.995174,0.909938,0.871658,1.000000,1.000000,0.777778,0.750000
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


<h3>
THESE ARE ALL THE FEATURE SETS I USED. TO STREAMLINE TESTING, I COMMENTED OUT PAST TEST SETS
</h3>

In [20]:
#feature_table = weather_table[['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed',
#                              'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
#                              'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainToday',
#                              'Year', 'Month']]

#feature_table = weather_table[['Pressure3pm', 'Humidity3pm', 'Rainfall']]

#feature_table = weather_table[['WindGustSpeed', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Year', 'Month']]

feature_table = weather_table[['Rainfall', 'Humidity3pm', 'RainToday']]

#feature_table = weather_table[['Temp9am', 'Temp3pm', 'MinTemp', 'MaxTemp', 
#           'Pressure9am', 'Pressure3pm', 'Humidity9am', 'Humidity3pm', 'WindSpeed9am', 'WindSpeed3pm']]

#creating feature set
feature_set = feature_table.values

#creating labels. Target is if it rained tomorrow
label_table = weather_table[['RainTomorrow']]
labels = label_table.values

<h3>
After ALOT of testing, I've found the most accurate feature set to be 'Rainfall', 'Humidity3pm', 'RainToday'
</h3>

In [21]:
feature_set.shape

(3117, 3)

In [22]:
labels.shape

(3117, 1)

<h3>
TRAINING  
</h3>

In [23]:
(weights, biases) = from_scratch_flex(feature_set, labels, hypers={'epochs':10, 'cost-reporting': 1})

(0, 0.12723251355326542)
(1, 0.11555442697231473)
(2, 0.11348999801465685)
(3, 0.11280541753902523)
(4, 0.11252024499420957)
(5, 0.1123876600405738)
(6, 0.11232264903246421)
(7, 0.11229024656355728)
(8, 0.11227439226539794)
(9, 0.11226715641785447)


In [24]:
predictions = ann_flex_tester(feature_set, weights, biases)

In [25]:
#featured = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed',
#                              'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
#                              'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainToday',
#                              'Year', 'Month']

#featured = ['WindGustSpeed', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Year', 'Month']

#featured = ['Pressure3pm', 'Humidity3pm', 'Rainfall']

featured = ['Rainfall', 'Humidity3pm', 'RainToday']

print("Influence Check:")
for w in range(len(weights)):
    print(featured[w],'\tWEIGHT: ', weights[w])
    #print(weights[w])
    
print()
print(biases)

print(predictions[:10])

Influence Check:
Rainfall 	WEIGHT:  [0.92119927]
Humidity3pm 	WEIGHT:  [9.88627604]
RainToday 	WEIGHT:  [0.48154709]

[-7.3711092]
[array([0.01513425]), array([0.00342417]), array([0.00686496]), array([0.00309976]), array([0.00309976]), array([0.00378241]), array([0.00378241]), array([0.00208111]), array([0.00280599]), array([0.00253999])]


In [26]:
acc = 0

for i in range(len(predictions)):
    pred = 0
    if (predictions[i] < 0.5):
        pred = 0
    if (predictions[i] >= 0.5):
        pred = 1
        
    if labels[i] == pred:
        acc += 1
        
my_accuracy = (acc/len(predictions))

my_accuracy

0.8354186717998076

<h3>
_______TEST_RESULT_RECORD________
</h3>

In [ ]:
'''
    
FEATURE_SET: 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed','WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am','Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainToday','Year', 'Month' 
F_S. SHAPE: (112925, 18) {The whole table}
EPOCHS: 1 (Even this takes > 5 minutes)
TOP INFLUENCERS: WindGustSpeed(16.5), Humidity3pm(12.5)
ACCURACY: 0.7588045162718619

*********************************************************************

FEATURE_SET: 'WindGustSpeed', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Year', 'Month'
F_S. SHAPE: (112925, 6)  {The whole table}
EPOCHS: 100 (Took awhile)
TOP INFLUENCERS:    WindGustSpeed 	WEIGHT:  [9.78464518]
                    Humidity3pm 	WEIGHT:  [9.67878535]
                    Pressure9am 	WEIGHT:  [4.57032098]
                    Pressure3pm 	WEIGHT:  [-84.11347502]
ACCURACY:  0.7834934691166704 (increase in accuracy/decrease in runtime)

*********************************************************************

FEATURE_SET: 'Pressure3pm', 'Humidity3pm', 'Rainfall'
F_S. SHAPE: (112925, 3)  {The whole table}
EPOCHS: 50 (Didn't take as long)
TOP INFLUENCERS:    Pressure3pm 	WEIGHT:  [-33.91523858]
                    Humidity3pm 	WEIGHT:  [7.98068426]
                    Rainfall 	WEIGHT:  [8.10093648]
ACCURACY:  0.7816426831968121 (decrease in accuracy/decrease in runtime)

*********************************************************************

FEATURE_SET: 'Rainfall', 'Humidity3pm', 'RainToday'
F_S. SHAPE: (112925, 6)  {The whole table}
EPOCHS: 50 (Didn't take as long)
TOP INFLUENCERS:    Rainfall 	WEIGHT:  [6.45114062]
                    Humidity3pm 	WEIGHT:  [8.8245798]
                    RainToday 	WEIGHT:  [0.90299662]
ACCURACY:  0.7468408235554571 (decrease in accuracy)

*********************************************************************

FEATURE_SET: 'Rainfall', 'Humidity3pm', 'RainToday'
F_S. SHAPE: (3117, 3)  {Only Data from the Darwin Weather Station}
EPOCHS: 50 (Quick)
TOP INFLUENCERS:    Rainfall 	WEIGHT:  [0.92119927]
                    Humidity3pm 	WEIGHT:  [9.88627604]
                    RainToday 	WEIGHT:  [0.48154709]
ACCURACY:  0.8354186717998076 (BIG increase in accuracy / BIG cut on runtime)

*********************************************************************

FEATURE_SET: 'Rainfall', 'Humidity3pm', 'RainToday'
F_S. SHAPE: (3117, 3)  {Only Data from the Darwin Weather Station}
EPOCHS: 1000 (Not as Quick)
TOP INFLUENCERS:    Rainfall 	WEIGHT:  [-1.48303605]
                    Humidity3pm 	WEIGHT:  [-6.87531263]
                    RainToday 	WEIGHT:  [0.61832885]
ACCURACY:  0.8594802694898941 (increase in accuracy with more epochs)

*********************************************************************

FEATURE_SET: 'Rainfall', 'Humidity3pm', 'RainToday'
F_S. SHAPE: (3117, 3)  {Only Data from the Darwin Weather Station}
EPOCHS: 85 (Very Quick)
TOP INFLUENCERS:    Rainfall 	WEIGHT:  [4.83360099]
                    Humidity3pm 	WEIGHT:  [-2.25044866]
                    RainToday 	WEIGHT:  [3.14936906]
ACCURACY:  0.8601219120949631 (slight increase in accuracy with less epochs)

*********************************************************************
'''

In [28]:
weather_table.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
135921,0.673401,0.871053,0.0,0.3125,0.325397,0.2500,0.5000,0.206349,0.326923,0.818182,0.323232,0.996762,0.996750,0.788820,0.863636,0,0,1.0,0.583333
135922,0.653199,0.852632,0.0,0.2500,0.396825,0.4375,0.3125,0.238095,0.538462,0.818182,0.171717,0.997547,0.997045,0.754658,0.852941,0,0,1.0,0.583333
135923,0.612795,0.836842,0.0,0.3125,0.365079,0.3750,0.2500,0.349206,0.365385,0.383838,0.242424,0.997940,0.997636,0.754658,0.834225,0,0,1.0,0.583333
135924,0.582492,0.807895,0.0,0.3125,0.349206,0.4375,0.3125,0.349206,0.250000,0.555556,0.161616,0.998038,0.998227,0.661491,0.796791,0,0,1.0,0.583333
135925,0.521886,0.810526,0.0,0.3750,0.365079,0.3125,0.3125,0.317460,0.365385,0.373737,0.161616,0.997057,0.997242,0.689441,0.791444,0,0,1.0,0.583333


In [29]:
weather_table.tail()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
139108,0.649832,0.878947,0.0,0.2500,0.277778,0.4375,0.1875,0.142857,0.384615,0.636364,0.323232,0.994702,0.995174,0.760870,0.863636,0,0,0.0,0.5
139109,0.713805,0.857895,0.0,0.3125,0.293651,0.4375,0.4375,0.206349,0.211538,0.565657,0.282828,0.995389,0.995864,0.770186,0.855615,0,0,0.0,0.5
139110,0.696970,0.863158,0.0,0.3125,0.261905,0.3125,0.8125,0.269841,0.211538,0.464646,0.232323,0.996076,0.996455,0.770186,0.858289,0,0,0.0,0.5
139111,0.656566,0.836842,0.0,0.3750,0.206349,0.4375,1.0000,0.142857,0.326923,0.626263,0.585859,0.995683,0.995371,0.770186,0.780749,0,0,0.0,0.5
139112,0.680135,0.834211,0.0,0.2500,0.238095,0.2500,1.0000,0.238095,0.134615,0.737374,0.323232,0.994702,0.994386,0.788820,0.828877,0,0,0.0,0.5
